In [1]:
!pip install transformers
from google.colab import drive
drive.mount('./mydata')
import sys
sys.path.append('./mydata/MyDrive/Colab Notebooks')

     |████████████████████████████████| 4.0 MB 4.6 MB/s 
     |████████████████████████████████| 6.6 MB 19.4 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 596 kB 49.6 MB/s 
     |████████████████████████████████| 880 kB 40.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d1e46199bbd182ad249abaac4099f625eccc0108c8227eefe1c18f863e166a3f
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Mounted at ./mydata


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [3]:
# coqa = pd.read_json('coqa-train-v1.0.json')
coqa = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
# coqa_dev = pd.read_json('coqa-dev-v1.0.json')
coqa.head()

,version,data
0,1,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,1,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,1,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,1,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."


In [4]:
### convert the whole thing into a one-trun QA setting

del coqa["version"]
#required columns in our dataframe
cols = ["text","question","answer"]
#list of lists to create our dataframe
comp_list = []
for index, row in coqa.iterrows():
    for i in range(len(row["data"]["questions"])):
        temp_list = []
        temp_list.append(row["data"]["story"])
        temp_list.append(row["data"]["questions"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["input_text"])
        comp_list.append(temp_list)

#saving the dataframe to csv file for further loading
new_df = pd.DataFrame(comp_list, columns=cols) 
new_df.to_csv("CoQA_data.csv", index=False)

In [5]:
# read the data again
data = pd.read_csv("CoQA_data.csv")
print("Number of question and answers: ", len(data))

Number of question and answers:  108647


In [6]:
# download the pretrained model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [7]:
def prediction(input_ids):
    # convert ids to tokens
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # for token, id in zip(tokens, input_ids):
    #     print('{:8}{:8,}'.format(token,id))

    #first occurence of [SEP] token
    # tokenizer.sep_token_id is the id of the seperator token [SEP]
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    # print("SEP token index: ", sep_idx)
    #number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
    num_seg_a = sep_idx+1
    # print("Number of tokens in segment A: ", num_seg_a)
    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a
    # print("Number of tokens in segment B: ", num_seg_b)
    #creating the segment ids as list consiting of 0s and 1s
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    #making sure that every input token has a segment id
    assert len(segment_ids) == len(input_ids)

    #token input_ids to represent the input and token segment_ids to differentiate our segments - question and text
    output = model(torch.tensor([input_ids]),  token_type_ids=torch.tensor([segment_ids]))

    #tokens with highest start and end scores
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)

    question = " ".join(tokens[:(num_seg_a)])
    question = question.replace('[cls]','').replace('[sep]','')
    print("\nQuestion:\n{}".format(question.capitalize()))

    if answer_end >= answer_start:
        answer = " ".join(tokens[answer_start:answer_end+1])
        print("Answer:\n{}.".format(answer.capitalize()))
    else:
        print("I am unable to find the answer to this question. Can you please ask another question?")
        

Try to attack the model on inputs (texts & questions)

In [21]:
# pick a random sample from the dataset
random_num = np.random.randint(0,len(data))
# or pick it by hand
random_num = 8575


question = data["question"][random_num]
text = data["text"][random_num]

print('question: '+question,'\n\n','text: '+text,'\n\n','answer: '+data["answer"][random_num])

# concatenate as inputs to the model
input_ids = tokenizer.encode(question, text)
print("\n The input has a total of {} tokens.".format(len(input_ids)))

question: Was the newsstand busy all day? 

 text: CHAPTER II 

A VISIT TO THE NAVY-YARD 

There was a rush of business at the news-stand between twelve and one o'clock, but shortly after one this died away, and inside of half an hour Phil Newell told Walter that they might be on their way--"If you are bound to enlist in Uncle Sam's service," he added. 

Walter made sure that the paper containing Job Dowling's permission for him to enter the navy was safe in his coat pocket, and then announced his readiness to depart. The owner of the stand called up Dan Brown and gave him a few directions, and in another minute Newell and Walter had boarded a Charlestown car and were off. 

"I haven't been over to the navy-yard for several years," remarked Phil Newell, as they rode along. "I used to know several of the boys that were there, but they've grown too old for the service. I reckon the yard is a busy place these days." 

And a busy place it proved to be as they turned into Chelsea Street, an

In [19]:
adversarial_text = text.replace('Budapest was two cities','Budapest was not one city')
#print(adversarial_text,'\n\n')

adversarial_question = question.replace('always ','')#+' for what?'
adversarial_question = 'Which asian city is more expensive ?'
#print(adversarial_question)

# tokenizer the question and text into word ids
# the tokens in question and text are seperated by '[SEP]' token
input_ids_t = tokenizer.encode(question, adversarial_text)
input_ids_q = tokenizer.encode(adversarial_question, text)

In [20]:
### run three predictions
prediction(input_ids)
prediction(input_ids_t)
prediction(input_ids_q)


Question:
[cls] was budapest always one city ? [sep]
Answer:
Budapest became one city in 1872.

Question:
[cls] was budapest always one city ? [sep]
Answer:
Budapest was not one city.

Question:
[cls] which asian city is more expensive ? [sep]
Answer:
Tokyo.


### Succeed example


1.   random_num = 75

      adversarial_question = 'What is the second largest city in the US famous for ?'

      *How to define this type of question?*


